Problem 1 part 1

In [ ]:
import numpy as np
x1 = np.array([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
x2 = np.array([1.0,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0.0])
p = np.array([28.1,34.3,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])
m = np.matmul(p.transpose(),x1)/(np.matmul(x1.transpose(),x1))
print(m)



problem 1 part 2

In [ ]:
import torch as t
from torch.autograd import Variable
import math as m

x = np.linspace(0,1,num=11)
x = t.from_numpy(x)
p_exp = np.array([28.1,34.3,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])
p_exp = t.from_numpy(p_exp)
A = Variable(t.tensor([1.0, 1.0]), requires_grad=True)
err = []
a = 0.01
for i in range(11):
    p[i] = x[i]*t.exp(A[0]*((A[1]*(1-x[i])/(A[0]*x[i]+A[1]*(1-x[i]))))**2)*17.4637\
        +(1-x[i])*t.exp(A[1]*((A[0]*x[i])/(A[0]*x[i]+A[1]*(1-x[i]))**2))*28.8241
    loss = ((x[i]*t.exp(A[0]*((A[1]*(1-x[i])/(A[0]*x[i]+A[1]*(1-x[i]))))**2)*17.4637\
        +(1-x[i])*t.exp(A[1]*((A[0]*x[i])/(A[0]*x[i]+A[1]*(1-x[i]))**2))*28.8241)\
            -p_exp[i])**2
    loss.backward()
    with t.no_grad():
        A -= a*A.grad
        A.grad.zero_()
        
print(A.data.numpy())
print(loss.data.numpy())


problem 2

In [3]:

import numpy as np
import matplotlib.pyplot as plt
def sample_loss(x):
    return (4-2.1*x[0]**2+(x[0]**4)/3)*x[0]**2 + x[0]*x[1] + (-4+4*x[1]**2)*x[1]**2
x1 = np.linspace(-3, 3, 60)
x2 = np.linspace(-2, 2, 40)
param_grid = np.array([[y1,y2] for y2 in x2 for y1 in x1])
real_loss = [sample_loss(x) for x in param_grid]
param_grid[np.array(real_loss).argmax(),:]

from matplotlib import rc

rc('text', usetex=True)
X1, X2 = np.meshgrid(x1,x2)
plt.figure()
cp = plt.contourf(X1, X2, np.array(real_loss).reshape(X1.shape))
plt.colorbar(cp)
plt.title('Filled contours plot of loss function $\mathcal{L}$($\gamma$, $C$)')
plt.xlabel('$C$')
plt.ylabel('$\gamma')
plt.show()

<Figure size 640x480 with 2 Axes>